In [1]:
import pandas as pd
import os
import sys
import dotenv
import pygbif.species as gbif
import pytaxize.gn as gn
import numpy as np
sys.path.append(os.getcwd())

dotenv.load_dotenv(".env")
data_dir = os.getenv("DATA_PATH")
#data_dir = "G:\GITAR\PubData"


# set data dir to "G:\GITAR\PubData"
#data_dir = r"G:\GITAR\PubData\\"

# Bring in new species lists


invasive_all_source = pd.read_csv( data_dir + "\species lists\invasive_all_source.csv", dtype={"usageKey": str})
gbif_backbone = pd.read_csv( data_dir + "\GBIF data\GBIF_backbone_invasive.csv", dtype={"usageKey": str})
cabi_gbif = pd.read_csv( data_dir + "\species lists\gbif_matched\cabi_gbif.csv", dtype={"usageKey": str})
eppo_gbif = pd.read_csv( data_dir + "\species lists\gbif_matched\eppo_gbif.csv", dtype={"usageKey": str})
daisie_gbif = pd.read_csv( data_dir + "\species lists\gbif_matched\daisie_gbif.csv", dtype={"usageKey": str})


In [33]:
#count ranks in invasive all source
invasive_all_source["rank"].value_counts()


rank
SPECIES       41775
GENUS          2062
SUBSPECIES     1420
VARIETY         763
FORM             74
FAMILY           65
KINGDOM          10
PHYLUM            6
CLASS             1
ORDER             1
Name: count, dtype: int64

In [2]:

data_files = [ cabi_gbif, eppo_gbif, daisie_gbif]


In [7]:
def get_species(row):

    sp_SINAS = row['speciesSINAS']
    sp_CABI = row['speciesCABI']
    sp_EPPO = row['speciesEPPO']
    sp_DAISIE = row['speciesDAISIE']
    sp_GBIF = row['speciesGBIF']


    

    if pd.notnull(row['speciesSINAS']):
        species = row['speciesSINAS']
    elif pd.notnull(row['speciesCABI']) :
        species = row['speciesCABI']
    
    elif pd.notnull(row['speciesEPPO']):
        species = row['speciesEPPO']
    elif pd.notnull(row['speciesDAISIE']):
        species = row['speciesDAISIE']
    else:
        species = None 
        
        print("No species found ")
    ######

    #search for species in gbif backbone
    search = gbif.name_suggest(q=species)


    return species

In [ ]:
def search_names(row):
    search = gbif.name_suggest(q=row['species'])
            
    if search['matchType'] == "HIGHERRANK" or search['matchType'] == "" or search['matchType'] == "NA" or search == []:
                search = gbif.name_lookup(row['species'], rank="SPECIES")
                if search['matchType'] == "SPECIES"


In [85]:
import pandas as pd
from pygbif import species
from tqdm import tqdm


def check_gbif_tax(dat):
    # Initialize new columns
    dat["scientificName"] = None
    dat["Taxon"] = dat["Taxon_orig"]
    dat["GBIFstatus"] = "Missing"
    dat["GBIFmatchtype"] = None
    dat["GBIFnote"] = None
    dat["GBIFstatus_Synonym"] = None
    dat["species"] = None
    dat["genus"] = None
    dat["family"] = None
    dat["class"] = None
    dat["order"] = None
    dat["phylum"] = None
    dat["kingdom"] = None
    dat["GBIFtaxonRank"] = None
    dat["GBIFusageKey"] = None

    # Determine taxlist based on columns available
    if "kingdom_user" in dat.columns:
        taxlist_lifeform = dat[["Taxon", "kingdom_user"]].drop_duplicates()
        taxlist = taxlist_lifeform["Taxon"].unique()
    elif "Author" in dat.columns:
        taxlist = (
            dat[["Taxon", "Author"]]
            .drop_duplicates()
            .apply(lambda x: " ".join(x), axis=1)
            .unique()
        )
    else:
        taxlist = dat["Taxon"].unique()

    n_taxa = len(taxlist)

    mismatches = pd.DataFrame(columns=["Taxon", "status", "matchType"])

    # Initialize progress bar
    tqdm.pandas()

    for j in tqdm(range(n_taxa), desc="Processing taxa"):
        taxon = taxlist[j]
        ind_tax = dat.index[dat["Taxon_orig"] == taxon]
        taxon = taxon.replace(" sp.", " ").replace(" spp.", " ").replace(" .f ", " ").replace(" .var", "")

        db_all = species.name_backbone(taxon, verbose=True, strict=True)
        db = {k: v for k, v in db_all.items() if k != "alternatives"}
        alternatives = db_all.get("alternatives", [])
        if db.get("status") == "ACCEPTED" and db.get("matchType") == "EXACT":
            dat.loc[ind_tax, "Taxon"] = db.get("canonicalName")
            dat.loc[ind_tax, "scientificName"] = db.get("scientificName")
            dat.loc[ind_tax, "GBIFstatus"] = db.get("status")
            dat.loc[ind_tax, "GBIFmatchtype"] = db.get("matchType")
            dat.loc[ind_tax, "GBIFtaxonRank"] = db.get("rank")
            dat.loc[ind_tax, "GBIFusageKey"] = db.get("usageKey")

            dat.loc[ind_tax, "species"] = db.get("species")
            dat.loc[ind_tax, "genus"] = db.get("genus")
            dat.loc[ind_tax, "family"] = db.get("family")
            dat.loc[ind_tax, "class"] = db.get("class")
            dat.loc[ind_tax, "order"] = db.get("order")
            dat.loc[ind_tax, "phylum"] = db.get("phylum")
            dat.loc[ind_tax, "kingdom"] = db.get("kingdom")
            continue

        elif (
            db.get("status") == "SYNONYM"
            and db.get("matchType") == "EXACT"
            and ("species" in db or 'genus' in db)
        ):
            dat.loc[ind_tax, "GBIFstatus"] = db.get("status")
            dat.loc[ind_tax, "GBIFmatchtype"] = db.get("matchType")
            dat.loc[ind_tax, "GBIFtaxonRank"] = db.get("rank")
            dat.loc[ind_tax, "GBIFusageKey"] = db.get("usageKey")

            if any(
                alt.get("status") == "ACCEPTED" and alt.get("matchType") == "EXACT"
                for alt in alternatives
            ):
                accepted_alt = next(
                    alt
                    for alt in alternatives
                    if alt.get("status") == "ACCEPTED"
                    and alt.get("matchType") == "EXACT"
                )
                dat.loc[ind_tax, "scientificName"] = accepted_alt.get("scientificName")
                dat.loc[ind_tax, "Taxon"] = accepted_alt.get("canonicalName")

                dat.loc[ind_tax, "species"] = accepted_alt.get("species")
                dat.loc[ind_tax, "genus"] = accepted_alt.get("genus")
                dat.loc[ind_tax, "family"] = accepted_alt.get("family")
                dat.loc[ind_tax, "class"] = accepted_alt.get("class")
                dat.loc[ind_tax, "order"] = accepted_alt.get("order")
                dat.loc[ind_tax, "phylum"] = accepted_alt.get("phylum")
                dat.loc[ind_tax, "kingdom"] = accepted_alt.get("kingdom")
                continue

            elif db.get("rank") == "SPECIES":
                dat.loc[ind_tax, "Taxon"] = db.get("species")
                dat.loc[ind_tax, "GBIFstatus"] = db.get("status")
                dat.loc[ind_tax, "GBIFmatchtype"] = db.get("matchType")
                dat.loc[ind_tax, "GBIFtaxonRank"] = db.get("rank")
                dat.loc[ind_tax, "GBIFusageKey"] = db.get("usageKey")

                db_all_2 = species.name_backbone(
                    dat.loc[ind_tax, "Taxon"].iloc[0], strict=True, verbose=True
                )
                db_2 = db_all_2

                if db_2.get("matchType") == "EXACT":
                    dat.loc[ind_tax, "scientificName"] = db_2.get("scientificName")
                    dat.loc[ind_tax, "GBIFstatus_Synonym"] = db_2.get("status")
                    dat.loc[ind_tax, "species"] = db_2.get("species")
                    dat.loc[ind_tax, "genus"] = db_2.get("genus")
                    dat.loc[ind_tax, "family"] = db_2.get("family")
                    dat.loc[ind_tax, "class"] = db_2.get("class")
                    dat.loc[ind_tax, "order"] = db_2.get("order")
                    dat.loc[ind_tax, "phylum"] = db_2.get("phylum")
                    dat.loc[ind_tax, "kingdom"] = db_2.get("kingdom")
            elif db.get("rank") == "GENUS":
                dat.loc[ind_tax, "Taxon"] = db.get("genus")
                dat.loc[ind_tax, "GBIFstatus"] = db.get("status")
                dat.loc[ind_tax, "GBIFmatchtype"] = db.get("matchType")
                dat.loc[ind_tax, "GBIFtaxonRank"] = db.get("rank")
                dat.loc[ind_tax, "GBIFusageKey"] = db.get("usageKey")

                db_all_2 = species.name_backbone(
                    dat.loc[ind_tax, "Taxon"].iloc[0], strict=True, verbose=True
                )
                db_2 = db_all_2

                if db_2.get("matchType") == "EXACT":
                    dat.loc[ind_tax, "scientificName"] = db_2.get("scientificName")
                    dat.loc[ind_tax, "GBIFstatus_Synonym"] = db_2.get("status")
                    dat.loc[ind_tax, "species"] = db_2.get("species")
                    dat.loc[ind_tax, "genus"] = db_2.get("genus")
                    dat.loc[ind_tax, "family"] = db_2.get("family")
                    dat.loc[ind_tax, "class"] = db_2.get("class")
                    dat.loc[ind_tax, "order"] = db_2.get("order")
                    dat.loc[ind_tax, "phylum"] = db_2.get("phylum")
                    dat.loc[ind_tax, "kingdom"] = db_2.get("kingdom")
                continue

        elif (
            db.get("status") == "ACCEPTED"
            and db.get("matchType") == "FUZZY"
            and db.get("confidence") == 100
        ):
            print("FUZZY")
            dat.loc[ind_tax, "Taxon"] = db.get("canonicalName")
            dat.loc[ind_tax, "scientificName"] = db.get("scientificName")
            dat.loc[ind_tax, "GBIFstatus"] = db.get("status")
            dat.loc[ind_tax, "GBIFmatchtype"] = db.get("matchType")
            dat.loc[ind_tax, "GBIFtaxonRank"] = db.get("rank")
            dat.loc[ind_tax, "GBIFusageKey"] = db.get("usageKey")

            dat.loc[ind_tax, "species"] = db.get("species")
            dat.loc[ind_tax, "genus"] = db.get("genus")
            dat.loc[ind_tax, "family"] = db.get("family")
            dat.loc[ind_tax, "class"] = db.get("class")
            dat.loc[ind_tax, "order"] = db.get("order")
            dat.loc[ind_tax, "phylum"] = db.get("phylum")
            dat.loc[ind_tax, "kingdom"] = db.get("kingdom")

        elif (
            db.get("matchType") == "HIGHERRANK"
            and db.get("status") == "ACCEPTED"
            
        ):
            if any(
                alt.get("status") in ['ACCEPTED', 'SYNONYM'] and alt.get("matchType") == "EXACT"
                for alt in alternatives
            ):
                accepted_alt = next(
                    alt
                    for alt in alternatives
                    if alt.get("status") in ['ACCEPTED', 'SYNONYM']
                    and alt.get("matchType") == "EXACT"
                )

                # if species in accepted alt
                if "species" in accepted_alt:
                    taxon = accepted_alt.get("species")
                elif "genus" in accepted_alt:
                    taxon = accepted_alt.get("genus")
                elif "family" in accepted_alt:
                    taxon = accepted_alt.get("family")

                db_all_2 = species.name_backbone(
                    taxon, strict=True, verbose=True
                )
                db_2 = db_all_2

                if db_2.get("matchType") == "EXACT":
                    dat.loc[ind_tax, "Taxon"] = db_2.get("canonicalName")
                    dat.loc[ind_tax, "scientificName"] = db_2.get("scientificName")
                    dat.loc[ind_tax, "GBIFstatus_Synonym"] = db_2.get("status")
                    dat.loc[ind_tax, "species"] = db_2.get("species")
                    dat.loc[ind_tax, "genus"] = db_2.get("genus")
                    dat.loc[ind_tax, "family"] = db_2.get("family")
                    dat.loc[ind_tax, "class"] = db_2.get("class")
                    dat.loc[ind_tax, "order"] = db_2.get("order")
                    dat.loc[ind_tax, "phylum"] = db_2.get("phylum")
                    dat.loc[ind_tax, "kingdom"] = db_2.get("kingdom")

            else:
                print('no accepted alt')
                #set to higher rank taxa
                dat.loc[ind_tax, "Taxon"] = db.get("canonicalName")
                dat.loc[ind_tax, "scientificName"] = db.get("scientificName")
                dat.loc[ind_tax, "GBIFstatus"] = db.get("status")
                dat.loc[ind_tax, "GBIFmatchtype"] = db.get("matchType")
                dat.loc[ind_tax, "GBIFtaxonRank"] = db.get("rank")
                dat.loc[ind_tax, "GBIFusageKey"] = db.get("usageKey")
                dat.loc[ind_tax, "species"] = db.get("species")
                dat.loc[ind_tax, "genus"] = db.get("genus")
                dat.loc[ind_tax, "family"] = db.get("family")
                dat.loc[ind_tax, "class"] = db.get("class")
                dat.loc[ind_tax, "order"] = db.get("order")
                dat.loc[ind_tax, "phylum"] = db.get("phylum")
                dat.loc[ind_tax, "kingdom"] = db.get("kingdom")
                print('falling back to higherrank')
                continue

        else:
            #print('checking last resort ' + taxon)
            last_resort_db =  species.name_backbone(taxon, verbose = True, strict = False)
            if last_resort_db.get("status") == "ACCEPTED" and last_resort_db.get("matchType") == "HIGHERRANK":
                dat.loc[ind_tax, "Taxon"] = last_resort_db.get("canonicalName")
                dat.loc[ind_tax, "scientificName"] = last_resort_db.get("scientificName")
                dat.loc[ind_tax, "GBIFstatus"] = last_resort_db.get("status")
                dat.loc[ind_tax, "GBIFmatchtype"] = last_resort_db.get("matchType")
                dat.loc[ind_tax, "GBIFtaxonRank"] = last_resort_db.get("rank")
                dat.loc[ind_tax, "GBIFusageKey"] = last_resort_db.get("usageKey")

                dat.loc[ind_tax, "species"] = last_resort_db.get("species")
                dat.loc[ind_tax, "genus"] = last_resort_db.get("genus")
                dat.loc[ind_tax, "family"] = last_resort_db.get("family")
                dat.loc[ind_tax, "class"] = last_resort_db.get("class")
                dat.loc[ind_tax, "order"] = last_resort_db.get("order")
                dat.loc[ind_tax, "phylum"] = last_resort_db.get("phylum")
                dat.loc[ind_tax, "kingdom"] = last_resort_db.get("kingdom")
            else:
                #print(taxon + "FAILED" )
                continue
        # Add mismatches to the mismatches dataframe using pd.concat()
        mismatch_entry = pd.DataFrame(
            [
                {
                    "Taxon": taxon,
                    "status": db.get("status"),
                    "matchType": db.get("matchType"),
                }
            ]
        )
        mismatches = pd.concat([mismatches, mismatch_entry], ignore_index=True)

    return dat, mismatches

In [13]:
#cycle through data files - collect all from species column where matchType is blank, NA or "HIGHERRANK" then make a dataframe for check_gbif_tax function with species names called species_to_recheck with Taxon and Taxon_orig cols

specie_unfound = []
for file in data_files:
    for index, row in file.iterrows():
        if row['matchType'] in ["", "NA", "HIGHERRANK"]:
            specie_unfound.append(row['species'])
#drop duplicates
specie_unfound = list(set(specie_unfound))

check_species_df = pd.DataFrame(specie_unfound, columns=["Taxon"])
#add Taxon_orig that duplicates Taxon col
check_species_df['Taxon_orig'] = check_species_df['Taxon']

In [86]:
matched_species, unmatched = check_gbif_tax(check_species_df)
#make dc into dataframe
matched_species = pd.DataFrame(matched_species)


Processing taxa:   0%|          | 0/19759 [00:00<?, ?it/s]

Processing taxa:  51%|█████     | 10030/19759 [3:34:41<3:06:33,  1.15s/it]

no accepted alt
falling back to higherrank


Processing taxa:  55%|█████▍    | 10779/19759 [3:50:07<2:20:28,  1.07it/s]

no accepted alt
falling back to higherrank


...

Processing taxa:  87%|████████▋ | 17163/19759 [6:03:15<50:16,  1.16s/it]   

no accepted alt
falling back to higherrank


Processing taxa:  97%|█████████▋| 19175/19759 [6:44:51<10:34,  1.09s/it]  

no accepted alt
falling back to higherrank


Processing taxa: 100%|██████████| 19759/19759 [6:57:18<00:00,  1.27s/it]


In [89]:
#if GBIFstatus == "Missing" or None and GBIFstatus_Synonym != None or "Missing", import Synonym status to GBIFstatus


def update_GBIFstatus(row):
    if row['GBIFstatus'] == "Missing" and row['GBIFstatus_Synonym'] != None:
        row['GBIFstatus'] = row['GBIFstatus_Synonym']
    elif row['GBIFstatus'] == None and row['GBIFstatus_Synonym'] != None:
        row['GBIFstatus'] = row['GBIFstatus_Synonym']
    return row

matched_species = matched_species.apply(update_GBIFstatus, axis=1)



In [90]:
"""other_sources = gn.resolve(row['species'])
                try:
                    matched_name = other_sources[0][0]
                    search = gbif.name_lookup(matched_name, rank="SPECIES")
                    if search['count'] == 0:
                        print('still no luck')"""

,Taxon,Taxon_orig,scientificName,GBIFstatus,GBIFmatchtype,GBIFnote,GBIFstatus_Synonym,species,genus,family,class,order,phylum,kingdom,GBIFtaxonRank,GBIFusageKey
0,Ammodendron,Ammodendron sp.,Ammodendron Fisch. ex DC.,ACCEPTED,EXACT,None,None,None,Ammodendron,Fabaceae,Magnoliopsida,Fabales,Tracheophyta,Plantae,GENUS,2971156.0
1,Criconemoides,Criconemella sp.,"Criconemoides Taylor, 1936",SYNONYM,EXACT,None,ACCEPTED,None,Criconemoides,Criconematidae,Chromadorea,Rhabditida,Nematoda,Animalia,GENUS,4281842.0
2,Coreus,Mesocerus sp.,"Coreus Fabricius, 1794",SYNONYM,EXACT,None,ACCEPTED,None,Coreus,Coreidae,Insecta,Hemiptera,Arthropoda,Animalia,GENUS,4388641.0
3,Nezara,Nezara graminea,"Nezara Amyot & Serville, 1843",ACCEPTED,HIGHERRANK,None,None,None,Nezara,Pentatomidae,Insecta,Hemiptera,Arthropoda,Animalia,GENUS,2078851.0
4,Thuemenidium,Thuemenidium sp.,Thuemenidium Kuntze,ACCEPTED,EXACT,None,None,None,Thuemenidium,Geoglossaceae,Geoglossomycetes,Geoglossales,Ascomycota,Fungi,GENUS,2586871.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19754,Rhyzobius,Rhyzobius sp.,"Rhyzobius Stephens, 1829",ACCEPTED,EXACT,None,None,None,Rhyzobius,Coccinellidae,Insecta,Coleoptera,Arthropoda,Animalia,GENUS,4990174.0
19755,Uromyces laburni f. sp. anglicae,Uromyces laburni f. sp. anglicae,None,Missing,None,None,None,None,None,None,None,None,None,None,None,NaN
19756,Dasyscyphus,Dasyscyphus sp.,"Dasyscyphus Nees ex Gray, 1821",ACCEPTED,EXACT,None,None,None,Dasyscyphus,Lachnaceae,Leotiomycetes,Helotiales,Ascomycota,Fungi,GENUS,2530392.0
19757,Araucaria,Araucaria sp.,Araucaria Juss.,ACCEPTED,EXACT,None,None,None,Araucaria,Araucariaceae,Pinopsida,Pinales,Tracheophyta,Plantae,GENUS,2684910.0


In [91]:
#write matched_species to csv as previously_unmatched_species.csv
matched_species.to_csv(data_dir + "\species lists\previously_unmatched_species_gbif_match.csv")

In [92]:
#combine data_files and matched specis. Add taxonomy columns to data_files.
for file in data_files:
    if "kingdom" not in file.columns:
        file["kingdom"] = None
        file["phylum"] = None
        file["class"] = None
        file["order"] = None
        file["family"] = None
        file["genus"] = None
        
        
        file["GBIFstatus"] = None
        file["GBIFtaxonRank"] = None
        file["species"] = None


    for index, row in file.iterrows():
        if row['matchType'] in ["", "NA", "HIGHERRANK"]:
            #search for species in matched_species
            search = matched_species[matched_species['Taxon_orig'] == row['species']]
            if search.empty:
                continue
            elif search.iloc[0]['GBIFstatus'] == "Missing" or search.iloc[0]['GBIFstatus'] == None:
                continue
            else:
                search = search.iloc[0]
            file.at[index, "kingdom"] = search['kingdom']
            file.at[index, "phylum"] = search['phylum']
            file.at[index, "class"] = search['class']
            file.at[index, "order"] = search['order']
            file.at[index, "family"] = search['family']
            file.at[index, "genus"] = search['genus']
            file.at[index, "usageKey"] = search['GBIFusageKey']
            file.at[index, "canonicalName"] = search['Taxon']
            file.at[index, "scientificName"] = search['scientificName']
            #file.at[index, "GBIFstatus"] = search['GBIFstatus']
            file.at[index, "matchType"] = search['GBIFmatchtype']
            file.at[index, "rank"] = search['GBIFtaxonRank']
            file.at[index, "species"] = search['species']

In [93]:
cabi_gbif

,species,codeCABI,source,usageKey,scientificName,canonicalName,rank,confidence,matchType,Date,New,kingdom,phylum,class,order,family,genus,GBIFstatus,GBIFtaxonRank
0,None,41703985,CABI,3112707,Cirsium wallichii DC.,Cirsium wallichii,SPECIES,98.0,EXACT,6/30/2021,False,None,None,None,None,None,None,None,None
1,None,28241311,CABI,1475957,"Alophora Robineau-Desvoidy, 1830",Alophora,GENUS,94.0,EXACT,6/30/2021,False,None,None,None,None,None,None,None,None
2,None,97635302,CABI,9652289,"Rhynocoris segmentarius (Germar, 1837)",Rhynocoris segmentarius,SPECIES,98.0,EXACT,6/30/2021,False,None,None,None,None,None,None,None,None
3,None,98769096,CABI,1703203,"Hieroglyphus tonkinensis I.Bolívar, 1912",Hieroglyphus tonkinensis,SPECIES,97.0,EXACT,6/30/2021,False,None,None,None,None,None,None,None,None
4,None,47065458,CABI,7265024,Haplocoelum Radlk.,Haplocoelum,GENUS,94.0,EXACT,6/30/2021,False,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12625,None,121709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-16,True,None,None,None,None,None,None,None,None
12626,None,62108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-16,True,None,None,None,None,None,None,None,None
12627,None,96034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-16,True,None,None,None,None,None,None,None,None
12628,None,110472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-16,True,None,None,None,None,None,None,None,None


In [54]:
test = gbif.name_backbone("Lycopersicon esculentum", strict=True, verbose=True)

In [30]:
# backbone where usageKey = 5060876

bb = gbif_backbone[gbif_backbone["usageKey"] == 3112707]

bb

,usageKey,scientificName,acceptedTaxonKey,acceptedScientificName,numberOfOccurrences,taxonRank,taxonomicStatus,kingdom,kingdomKey,phylum,...,classKey,order,orderKey,family,familyKey,genus,genusKey,species,speciesKey,iucnRedListCategory


In [111]:
row

species               albuca
codeCABI            44007809
source                   NaN
usageKey               XXalb
scientificName           NaN
canonicalName            NaN
rank                     NaN
confidence               NaN
matchType                NaN
Date              2023-05-04
New                    False
kingdom                     
phylum                      
class                       
order                       
family                      
genus                       
Name: 12099, dtype: object

In [12]:
gbif.name_lookup("Lycopersicon esculentum", rank="SPECIES")["results"][0]

{'key': 159142211,
 'nameKey': 6530446,
 'datasetKey': '02fff9ea-e629-458d-b99e-cb7dc9deae01',
 'nubKey': 2930181,
 'parentKey': 159142210,
 'parent': 'Lycopersicon',
 'kingdom': 'Plantae',
 'phylum': 'Tracheophyta',
 'order': 'Solanales',
 'family': 'Solanaceae',
 'genus': 'Lycopersicon',
 'species': 'Lycopersicon esculentum',
 'kingdomKey': 159140699,
 'phylumKey': 159140751,
 'classKey': 159141015,
 'orderKey': 159142130,
 'familyKey': 159142176,
 'genusKey': 159142210,
 'speciesKey': 159142211,
 'scientificName': 'Lycopersicon esculentum',
 'canonicalName': 'Lycopersicon esculentum',
 'authorship': '',
 'accordingTo': 'Taxonomía tomada de: Species 2000 & ITIS Catalogue of Life: April 2013; Estado taxonómico tomado de: http://www.iplantcollaborative.org/',
 'nameType': 'SCIENTIFIC',
 'taxonomicStatus': 'ACCEPTED',
 'rank': 'SPECIES',
 'origin': 'SOURCE',
 'numDescendants': 0,
 'numOccurrences': 0,
 'habitats': [],
 'nomenclaturalStatus': [],
 'threatStatuses': [],
 'descriptions': [

In [21]:
def get_sp_id(sp):
    sp_dict = gbif.name_backbone(sp)
    if "speciesKey" in sp_dict:
        return sp_dict["speciesKey"], sp_dict["species"]
    else:
        return None, None

In [24]:
gbif.name_backbone(name="Solanum verbascifolium")

{'usageKey': 2928997,
 'scientificName': 'Solanum L.',
 'canonicalName': 'Solanum',
 'rank': 'GENUS',
 'status': 'ACCEPTED',
 'confidence': 97,
 'matchType': 'HIGHERRANK',
 'kingdom': 'Plantae',
 'phylum': 'Tracheophyta',
 'order': 'Solanales',
 'family': 'Solanaceae',
 'genus': 'Solanum',
 'kingdomKey': 6,
 'phylumKey': 7707728,
 'classKey': 220,
 'orderKey': 1176,
 'familyKey': 7717,
 'genusKey': 2928997,
 'synonym': False,
 'class': 'Magnoliopsida'}

In [22]:
get_synonyms(get_sp_id("Lycopersicon esculentum"))

HTTPError: 400 Client Error: Bad Request for url: https://api.gbif.org/v1/species/(None,%20None)/synonyms?limit=100

In [23]:
get_sp_id("Lycopersicon esculentum")

(None, None)

In [16]:
def get_synonyms(sp_key):
    nu = gbif.name_usage(sp_key, data="synonyms")["results"]
    if len(nu) > 0:
        return [i["canonicalName"] for i in nu if "canonicalName" in i]
    else:
        return []

In [25]:
 search = gbif.name_backbone(name=row['species'])

            if search['matchType'] == "NONE":
                search = gbif.name_lookup(row['species'], rank="SPECIES")
            if 'count' in search and search['count'] == 0:
                print('gn resolving' + row['species'])
                other_sources = gn.resolve(row['species'])
                try:
                    matched_name = other_sources[0][0]
                    search = gbif.name_lookup(matched_name, rank="SPECIES")
                    if search['count'] == 0:
                        print('still no luck')
                        continue
                    search = search['results'][0]
                    try:
                        row['kingdom'] = search['kingdom']
                    except KeyError:
                        pass
                    try:
                        row['phylum'] = search['phylum']
                    except KeyError:
                        pass
                    try:
                        row['class'] = search['class']
                    except KeyError:
                        pass
                    try:

                        row['order'] = search['order']
                    except KeyError:
                        pass
                    try:
                        row['family'] = search['family']
                    except KeyError:
                        pass
                    try:
                        row['genus'] = search['genus']
                        row['usageKey'] = search['usageKey']
                        #row['matchType'] = search['matchType']
                        row['canonicalName'] = search['canonicalName']
                        row['scientificName'] = search['scientificName']
                        print("changed row" + row['species'] + " to " + search['scientificName'] + " with usageKey " + search['usageKey'])
                    except KeyError:
                        print('HR KeyError 1')
                        continue
                except IndexError:
                    print("HR IndexError")
                    continue

            else:
                if 'results' in search:
                    search = search['results'][0]

                try:
                    row['kingdom'] = search['kingdom']
                except KeyError:
                    pass
                try:
                    row['phylum'] = search['phylum']
                except KeyError:
                    pass
                try:
                    row['class'] = search['class']
                except KeyError:
                    pass
                try:

                    row['order'] = search['order']
                except KeyError:
                    pass
                try:
                    row['family'] = search['family']
                except KeyError:
                    pass
                try:
                    row['usageKey'] = search['usageKey']
                except KeyError:
                    try: 
                        row['usageKey'] = search['speciesKey']
                    except KeyError:
                        pass
                try:
                    row['genus'] = search['genus']
                    
                    #row['matchType'] = search['matchType']
                    row['canonicalName'] = search['canonicalName']
                    row['scientificName'] = search['scientificName']
                    print("changed row" + row['species'] + " to " + search['scientificName'] + " with usageKey ")
                except KeyError:
                    print('HR KeyErro 2')
                    
                    continue

IndentationError: unexpected indent (39153588.py, line 3)

In [81]:
search3['count']

KeyError: 'count'

In [64]:
test = gbif.name_lookup("Trientalis europaea", rank="SPECIES")["results"][0]
test

{'key': 206098550,
 'nameKey': 11447999,
 'datasetKey': '2c38cf8a-f981-4dfb-bc9d-dd2b6fc792ed',
 'nubKey': 3169295,
 'parentKey': 206113510,
 'parent': 'Plantae',
 'kingdom': 'Plantae',
 'species': 'Trientalis europaea',
 'kingdomKey': 206113510,
 'speciesKey': 206098550,
 'scientificName': 'Trientalis europaea',
 'canonicalName': 'Trientalis europaea',
 'authorship': '',
 'nameType': 'SCIENTIFIC',
 'taxonomicStatus': 'ACCEPTED',
 'rank': 'SPECIES',
 'origin': 'SOURCE',
 'numDescendants': 0,
 'numOccurrences': 0,
 'habitats': [],
 'nomenclaturalStatus': [],
 'threatStatuses': [],
 'descriptions': [],
 'vernacularNames': [],
 'higherClassificationMap': {'206113510': 'Plantae'},
 'synonym': False}

In [93]:
test2 = gbif.name_suggest(q="Athyrium alpestre")

In [105]:
test2[2]

{'key': 8282577,
 'nameKey': 1166416,
 'kingdom': 'Plantae',
 'phylum': 'Tracheophyta',
 'order': 'Polypodiales',
 'family': 'Athyriaceae',
 'genus': 'Pseudathyrium',
 'species': 'Pseudathyrium alpestre',
 'kingdomKey': 6,
 'phylumKey': 7707728,
 'classKey': 7228684,
 'orderKey': 392,
 'familyKey': 4922264,
 'genusKey': 7289277,
 'speciesKey': 4017779,
 'parent': 'Pseudathyrium',
 'parentKey': 7289277,
 'nubKey': 8282577,
 'scientificName': 'Athyrium alpestre (Hoppe) Rylands ex T.Moore',
 'canonicalName': 'Athyrium alpestre',
 'rank': 'SPECIES',
 'status': 'HETEROTYPIC_SYNONYM',
 'higherClassificationMap': {'6': 'Plantae',
  '7707728': 'Tracheophyta',
  '7228684': 'Polypodiopsida',
  '392': 'Polypodiales',
  '4922264': 'Athyriaceae',
  '7289277': 'Pseudathyrium',
  '4017779': 'Pseudathyrium alpestre'},
 'synonym': True,
 'class': 'Polypodiopsida'}

In [106]:
test3 = gbif.name_backbone(name="Athyrium alpestre", strict=True)
test3

{'usageKey': 4922264,
 'scientificName': 'Athyriaceae',
 'canonicalName': 'Athyriaceae',
 'rank': 'FAMILY',
 'status': 'ACCEPTED',
 'confidence': 98,
 'matchType': 'HIGHERRANK',
 'kingdom': 'Plantae',
 'phylum': 'Tracheophyta',
 'order': 'Polypodiales',
 'family': 'Athyriaceae',
 'kingdomKey': 6,
 'phylumKey': 7707728,
 'classKey': 7228684,
 'orderKey': 392,
 'familyKey': 4922264,
 'synonym': False,
 'class': 'Polypodiopsida'}

In [33]:
#make a list of all species in the invasive_all_source list that do not have a usageKey and have a canonicalName beginning with "xx" or "XX"
xxlist = []
for index, row in invasive_all_source.iterrows():

#if usageKey begins with "xx" or "XX" a
    if row['usageKey'].startswith("xx") or row['usageKey'].startswith("XX"):
      
        if pd.notnull(row['speciesCABI']):
            species = row['speciesCABI']
        elif pd.notnull(row['speciesASFR']):
            species = row['speciesASFR']
        
        elif pd.notnull(row['speciesEPPO']):
            species = row['speciesEPPO']
        elif pd.notnull(row['speciesDAISIE']):
            species = row['speciesDAISIE']
        else:
            species = "NA"
        xxlist.append(species)

In [13]:
invasive_all_source
#add columns to invasive_all_source for kingdom, phylum, class, order, family, genus
invasive_all_source["kingdom"] = ""
invasive_all_source["phylum"] = ""
invasive_all_source["class"] = ""
invasive_all_source["order"] = ""
invasive_all_source["family"] = ""
invasive_all_source["genus"] = ""


,speciesCABI,codeCABI,usageKey,invasiveCABI,speciesASFR,source,speciesEPPO,codeEPPO,invasiveEPPO,speciesDAISIE,codeDAISIE,scientificName,canonicalName,rank
0,Fusarium solani,81903.0,5251899,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fusarium solani W.C.Snyder,Fusarium solani,SPECIES
1,Acridotheres cristatellus,97370105.0,2489010,False,Acridotheres cristatellus,ASFR,NaN,NaN,NaN,Acridotheres cristatellus,50310.0,"Acridotheres cristatellus (Linnaeus, 1758)",Acridotheres cristatellus,SPECIES
2,Macrorhynchia philippina,53061963.0,2266916,False,Macrorhynchia philippina,ASFR,NaN,NaN,NaN,Macrorhynchia philippina,100544.0,"Macrorhynchia philippina Kirchenpauer, 1872",Macrorhynchia philippina,SPECIES
3,Peronospora aquilegiicola,12131388.0,2583264,True,NaN,NaN,Peronospora sp.,PEROSP,False,NaN,NaN,Peronospora Corda,Peronospora,GENUS
4,Peronospora aquilegiicola,12131388.0,2583264,True,NaN,NaN,Peronospora aquilegiicola,PEROAQ,False,NaN,NaN,Peronospora Corda,Peronospora,GENUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52989,NaN,NaN,3221935,NaN,NaN,NaN,NaN,NaN,NaN,Brenneria quercina,901589.0,"Brenneria quercina (Hildebrand & Schroth, 1967...",Brenneria quercina,SPECIES
52990,NaN,NaN,3221940,NaN,NaN,NaN,NaN,NaN,NaN,Erwinia alni,901591.0,"Erwinia alni Surico et al., 1996",Erwinia alni,SPECIES
52991,NaN,NaN,3221944,NaN,NaN,NaN,NaN,NaN,NaN,Erwinia salicis,901593.0,"Erwinia salicis (Day, 1924) Chester, 1939",Erwinia salicis,SPECIES
52992,NaN,NaN,2591578,NaN,NaN,NaN,NaN,NaN,NaN,Myriellina cydoniae,901601.0,Myriellina cydoniae (Desm.) Höhn.,Myriellina cydoniae,SPECIES
